In [57]:
import pandas as pd

In [ ]:
census_data = pd.read_csv('csv files/population census.csv')
missing_data = census_data.isnull().sum()
print("Missing data in each column:\n", missing_data)
print(census_data.dtypes)

Missing data in each column:
 Unnamed: 0                                       0
Unnamed: 1                                       0
Population and dwelling counts (13)              0
Population, 2021                                 0
Population, 2016                                 0
Population percentage change, 2016 to 2021       0
Land area in square kilometres, 2021             0
Population density per square kilometre, 2021    0
dtype: int64
Unnamed: 0                                        object
Unnamed: 1                                        object
Population and dwelling counts (13)               object
Population, 2021                                  object
Population, 2016                                  object
Population percentage change, 2016 to 2021       float64
Land area in square kilometres, 2021              object
Population density per square kilometre, 2021     object
dtype: object


In [ ]:
census_data['Population, 2021'] = pd.to_numeric(census_data['Population, 2021'].astype(str).str.replace(',', ''))                                               # Remove commas and convert to numeric data types
census_data['Population, 2016'] = pd.to_numeric(census_data['Population, 2016'].astype(str).str.replace(',', '').str.replace('r', ''))
census_data['Land area in square kilometres, 2021'] = pd.to_numeric(census_data['Land area in square kilometres, 2021'].astype(str).str.replace(',', ''))
census_data['Population density per square kilometre, 2021'] = pd.to_numeric(census_data['Population density per square kilometre, 2021'].astype(str).str.replace(',', ''))


census_data.rename(columns={                                                                # Rename columns for easier access
    'Population, 2021': 'population_2021',
    'Population, 2016': 'population_2016',
    'Population percentage change, 2016 to 2021': 'population_change_2016_2021',
    'Land area in square kilometres, 2021': 'land_area_km2',
    'Population density per square kilometre, 2021': 'population_density_km2'
}, inplace=True)


In [60]:
odhf_cleaned = pd.read_csv('cleaned csv files/odhf_cleaned.csv')

unique_odhf_provinces = odhf_cleaned['province'].unique()
unique_cenus_provinces = census_data['Population and dwelling counts (13)'].unique()
print("odhf abbreviations:",unique_odhf_provinces,"population cenus abbreviations:", unique_cenus_provinces)
      


odhf abbreviations: ['ab' 'bc' 'mb' 'nb' 'nl' 'ns' 'nt' 'nu' 'on' 'pe' 'qc' 'sk' 'yt'] population cenus abbreviations: ['Province or territory abbreviation' '...' 'N.L.' 'P.E.I.' 'N.S.' 'N.B.'
 'N.B. / Que.' 'Que.' 'Ont.' 'Ont. / Que.' 'Man.' 'Sask.' 'Alta.'
 'Alta. / Sask.' 'B.C.' 'Y.T.' 'N.W.T.' 'Nvt.']


In [61]:
abbreviation_normalization = {
    'N.L.':     'nl',
    'P.E.I.':    'pe',
    'N.S.':      'ns',
    'N.B.':      'nb',
    'Que.':     'qc',
    'Ont.':     'on',
    'Man.':     'mb',
    'Sask.':    'sk',
    'Alta.':    'ab',
    'B.C.':     'bc',
    'Y.T.':     'yt',
    'N.W.T.':   'nt',
    'Nvt.':     'nu'
}

census_data['Population and dwelling counts (13)'] = census_data['Population and dwelling counts (13)'].replace(abbreviation_normalization)       # normalizing the provinces column between both databases

multi_province_rows = census_data['Population and dwelling counts (13)'].str.contains('/', na=False)                                              # deleting entries that contain aggregated data from cities apart of 2 or more provinces
census_data = census_data[~multi_province_rows]

In [ ]:
census_data.rename(columns={'Population and dwelling counts (13)': 'province'}, inplace=True)                                                     # renaming the 'Population and dwelling counts (13)' to 'province' to match the odhf_cleaned.csv index column name

census_data.to_csv('population_census_cleaned.csv', index=False)

In [ ]:
bc_population_sum = census_data[census_data['province'] == 'bc']['population_2021'].sum()  ## Sum the 'population_2021' values for rows where 'province' is 'bc'

print(bc_population_sum)

9478627


In [ ]:
test_df = pd.read_csv('csv files/population census.csv')
test_df['Population, 2021'] = pd.to_numeric(test_df['Population, 2021'].astype(str).str.replace(',', '').str.replace('r', ''))   # same as above but using the original population census csv file to confirm previous result
bc_population_sum = test_df[test_df['Population and dwelling counts (13)'] == 'B.C.']['Population, 2021'].sum()


print(bc_population_sum)

9478627
